# Make input data for ml models

## graph data to ml data

In [1]:
import numpy as np
import pandas as pd
import os
import pickle as pkl

In [11]:
def graph2mlData(dataset='TEV_WT', feature='seq', split=2):
    idx = pkl.load(open('/scratch/cl1205/protease-gcnn-pytorch/data/ind.{}_binary_10_ang_aa_energy_7_energyedge_5_hbond.x'.format(dataset), 'rb'))
    idy = pkl.load(open('/scratch/cl1205/protease-gcnn-pytorch/data/ind.{}_binary_10_ang_aa_energy_7_energyedge_5_hbond.y'.format(dataset), 'rb'))
    test_index = np.loadtxt('/scratch/cl1205/protease-gcnn-pytorch/data/ind.{}_binary_10_ang_aa_energy_7_energyedge_5_hbond.test.index'.format(dataset), dtype=int)
    sequences = pkl.load(open('/scratch/cl1205/protease-gcnn-pytorch/data/ind.{}_binary_10_ang_aa_energy_7_energyedge_5_hbond.sequences'.format(dataset), 'rb'))
    if split == 2:
        test_index = np.loadtxt('/scratch/cl1205/protease-gcnn-pytorch/data/ind.{}_binary_10_ang_aa_energy_7_energyedge_5_hbond.test.index'.format(dataset), dtype=int)
        test_index = np.sort(test_index)
        X_test = idx[test_index]
        y_test = idy[test_index]
        train_mask = np.array([i not in test_index for i in range(idx.shape[0])]) 
    elif split == 3:
        test_index = np.loadtxt('/scratch/cl1205/protease-gcnn-pytorch/data/ind.{}_binary_10_ang_aa_energy_7_energyedge_5_hbond.trisplit.test.index'.format(dataset), dtype=int)
        val_index =  np.loadtxt('/scratch/cl1205/protease-gcnn-pytorch/data/ind.{}_binary_10_ang_aa_energy_7_energyedge_5_hbond.trisplit.val.index'.format(dataset), dtype=int)
        val_index = np.sort(val_index)
        X_val = idx[val_index]
        y_val = idy[val_index]
        test_index = np.sort(test_index)
        X_test = idx[test_index]
        y_test = idy[test_index]
        train_mask = np.array([i not in test_index and i not in val_index for i in range(idx.shape[0])]) 

    X_train = idx[train_mask]
    y_train = idy[train_mask]
    seq_train = np.array(sequences)[train_mask]
    seq_test = np.array(sequences)[test_index]
    seq_val = np.array(sequences)[val_index]

    if dataset == 'TEV_WT':
        if feature == 'seq':
            X_test = pd.DataFrame(graph2seq(X_test, length=8), index=[x[1:6] for x in seq_test])
            X_train = pd.DataFrame(graph2seq(X_train, length=8), index=[x[1:6] for x in seq_train])
            if split == 3:
                X_val = pd.DataFrame(graph2seq(X_val, length=8), index=[x[1:6] for x in seq_val])
    elif dataset == 'TEV_all' or dataset.find('HCV') != -1:
#         tmp_test = pd.DataFrame(num2hot4tevall(seq_test), index=[x.split('_')[1][1:6] for x in seq_test])
#         tmp_train = pd.DataFrame(num2hot4tevall(seq_train), index=[x.split('_')[1][1:6] for x in seq_train])
        tmp_test = pd.DataFrame()
        tmp_train = pd.DataFrame()
        if split == 3:
            tmp_val = pd.DataFrame() 
        if feature == 'seq':
            X_test = pd.DataFrame(graph2seq(X_test, length=5, special=dataset), index=[x.split('_')[1][1:6] for x in seq_test])
            X_train = pd.DataFrame(graph2seq(X_train, length=5,special=dataset), index=[x.split('_')[1][1:6] for x in seq_train])
            if split == 3:
                X_val = pd.DataFrame(graph2seq(X_val, length=5,special=dataset), index=[x.split('_')[1][1:6] for x in seq_val])
        elif feature == 'flatten':
            graph = pkl.load(open('/scratch/cl1205/protease-gcnn-pytorch/data/ind.{}_binary_10_ang_aa_energy_7_energyedge_5_hbond.graph'.format(dataset), 'rb'))
            graph_train = graph[train_mask]
            graph_test = graph[test_index]
            if split == 3:
                graph_val = graph[val_index] 
            X_test = pd.DataFrame(graph2flattened(X_test, graph_test), index=seq_test) #[x.split('_')[1][1:6] for x in seq_test])
            X_train = pd.DataFrame(graph2flattened(X_train, graph_train), index=seq_train) #[x.split('_')[1][1:6] for x in seq_train])
            if split == 3:
                X_val = pd.DataFrame(graph2flattened(X_val, graph_val), index=seq_val)
#         X_test = pd.concat((tmp_test, X_test), axis=1)
#         X_train = pd.concat((tmp_train, X_train), axis=1)
#         if split == 3:
#             X_val = pd.concat((tmp_val, X_val), axis=1)
        
    y_train = binary2labelY(y_train)
    y_test = binary2labelY(y_test)
    if split == 3:
        y_val = binary2labelY(y_val)
    
    y_test = pd.DataFrame(y_test, index=[x[1:7] for x in seq_test])
    y_train = pd.DataFrame(y_train, index=[x[1:7] for x in seq_train])
    if split == 3:
        y_val = pd.DataFrame(y_val, index=[x[1:7] for x in seq_val])
    
    if feature == 'flatten':
        dataset = dataset + '_10_ang_aa_energy_7_energyedge_5_hbond'
        
    if split == 2:
        pkl.dump(X_train, open('/scratch/cl1205/ml-cleavage/Data/X_train_{}_noProtID_binary_0.7'.format(dataset),'wb'))
        pkl.dump(X_test, open('/scratch/cl1205/ml-cleavage/Data/X_test_{}_noProtID_binary_0.3'.format(dataset),'wb'))

        pkl.dump(y_train, open('/scratch/cl1205/ml-cleavage/Data/y_train_{}_noProtID_binary_0.7'.format(dataset), 'wb'))
        pkl.dump(y_test, open('/scratch/cl1205/ml-cleavage/Data/y_test_{}_noProtID_binary_0.3'.format(dataset),'wb'))
    elif split == 3:
        pkl.dump(X_train, open('/scratch/cl1205/ml-cleavage/Data/X_train_{}_noProtID_trisplit_0.8'.format(dataset),'wb'))
        pkl.dump(X_test, open('/scratch/cl1205/ml-cleavage/Data/X_test_{}_noProtID_trisplit_0.1'.format(dataset),'wb'))
        pkl.dump(X_val, open('/scratch/cl1205/ml-cleavage/Data/X_val_{}_noProtID_trisplit_0.1'.format(dataset),'wb')) 

        pkl.dump(y_train, open('/scratch/cl1205/ml-cleavage/Data/y_train_{}_noProtID_trisplit_0.8'.format(dataset), 'wb'))
        pkl.dump(y_test, open('/scratch/cl1205/ml-cleavage/Data/y_test_{}_noProtID_trisplit_0.1'.format(dataset),'wb'))
        pkl.dump(y_val, open('/scratch/cl1205/ml-cleavage/Data/y_val_{}_noProtID_trisplit_0.1'.format(dataset),'wb')) 

    

In [ ]:
#"ACDEFGHIKLMNPQRSTVWY"

In [5]:
def num2hot4tevall(seqs):
    variants_list = {v:u for u,v in enumerate(['E107D_D127A_S135F', 'WT', 'H28L_T30A', 
                                                   'T17S_N68D_E107D_D127A_F132L_S135F_F162S_K229E',
                                                   'N176I', 'V209M_W211I_M218F', 'L2F', 
                                                   'I138T_N171D_N176T', 'T146S_D148P_S153N_S170A_N177M',
                                                   'E107D_D127A_S135F_R203Q_K215E'])}
    feat = np.zeros((len(seqs), len(variants_list)))
    for i, seq in enumerate(seqs):
        var = ('_').join(item.split('_')[0:-1])
        feat[i, variants_list[var]] = 1
    return feat

In [3]:
def binary2labelY(data):
    newData = np.empty(data.shape[0], dtype=object)
    for i in range(data.shape[0]):
        if data[i,0] == 1:
            newData[i] = 'CLEAVED'
        elif data[i,1] == 1:
            newData[i] = 'UNCLEAVED'
    return newData

In [7]:
def graph2seq(data, length,special=None):
    """
    Pick out the first 20 columns for each row
    """
    if special == None:
        truncate = data[:,0:length,0:20]
    elif special == 'TEV_all':
        truncate = data[:,np.r_[0:5,-3:-1],0:20]
    final = np.array([truncate[i,:,:].reshape(1,-1)[0] for i in range(truncate.shape[0])])
    return final

In [2]:
def graph2flattened(data, tgraph):
    """
    flatten graph features
    """
    return np.concatenate((data.reshape((data.shape[0],-1)), tgraph.reshape((tgraph.shape[0],-1))), axis=1)

In [ ]:
graph2mlData('TEV_all', 'seq')

In [ ]:
graph2mlData('TEV_all', 'flatten')

In [9]:
def label2onehot(data, n_classes):
    try:
        data = data[0].values
    except:
        pass
    newData = np.empty((data.shape[0], n_classes), dtype=object)
    for i in range(data.shape[0]):
        if data[i] == 'CLEAVED':
            newData[i] = [1, 0]
        elif data[i] == 'UNCLEAVED':
            newData[i] = [0,1]
    return newData

In [12]:
for data in ['HCV_all', 'HCV_WT','HCV_A171T','HCV_D183A','HCV_Triple']:
    graph2mlData(data, 'flatten', 3)

In [ ]:
for data in ['TEV_all']:
    graph2mlData(data, 'flatten', 3)

In [23]:
def testgraph2mlData(dataset='test_tev_design_20220912_binary_10_ang_aa_energy_7_coord_energyedge_5_hbond'):
    X_test = pkl.load(open('/scratch/cl1205/protease-gcnn-pytorch/data/ind.{}.x'.format(dataset), 'rb'))
    y_test = pkl.load(open('/scratch/cl1205/protease-gcnn-pytorch/data/ind.{}.y'.format(dataset), 'rb'))
    seq_test = pkl.load(open('/scratch/cl1205/protease-gcnn-pytorch/data/ind.{}.sequences'.format(dataset), 'rb'))
    seq_test = np.array(seq_test)
    graph_test = pkl.load(open('/scratch/cl1205/protease-gcnn-pytorch/data/ind.{}.graph'.format(dataset), 'rb'))
    X_test = pd.DataFrame(graph2flattened(X_test, graph_test), index=seq_test)
    y_test = binary2labelY(y_test)
    y_test = pd.DataFrame(y_test, index=[x.split('.')[0] for x in seq_test])
    pkl.dump(X_test, open('/scratch/cl1205/ml-cleavage/Data/X_test_{}'.format(dataset),'wb'))
    pkl.dump(y_test, open('/scratch/cl1205/ml-cleavage/Data/y_test_{}'.format(dataset),'wb'))


In [24]:
testgraph2mlData('test_tev_design_20220912_binary_10_ang_aa_energy_7_coord_energyedge_5_hbond')

In [25]:
testgraph2mlData('test_tev_design_p1pA_20220911_binary_10_ang_aa_energy_7_energyedge_5_hbond')

In [26]:
dataset = 'test_tev_design_20220912_binary_10_ang_aa_energy_7_coord_energyedge_5_hbond'
X_test = pkl.load(open('/scratch/cl1205/ml-cleavage/Data/X_test_{}'.format(dataset), 'rb'))
y_test = pkl.load(open('/scratch/cl1205/ml-cleavage/Data/y_test_{}'.format(dataset), 'rb'))

# Train and Test ML models

In [3]:
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
import tensorflow as tf
from tensorflow import keras
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import os
import pickle as pkl
from sklearn import preprocessing

/home/cl1205/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/cl1205/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/cl1205/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/cl1205/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: 

In [4]:
os.chdir('/scratch/cl1205/ml-cleavage/scripts')
from utils import *

In [5]:
def trainSeqOnly(dataset, save = '/scratch/cl1205/ml-cleavage/outputs/seqOnly_20220217', model = 'logistic_regression',
                encoding = 'energy', split=2, testset = None, mode = 'train', lr=None, dropout=None):
    classes = 2
    if split == 2:
        X_train, y_train, X_test, y_test = load_data(dataset, classes)
    elif split == 3:
        X_train, y_train, X_val, y_val, X_test, y_test = load_data(dataset, classes, 3)
    if testset != None:
        X_test = pkl.load(open('../Data/X_test_' + testset,'rb'))
        y_test = pkl.load(open('../Data/y_test_' + testset,'rb'))
        
    energy_indices = []
    seq_indices = []
    if dataset.find('TEV_all') != -1:
        for i in range(X_train.shape[1]):
            if i >= 1316: #1326:
                energy_indices.append(i)
            else:
                if i % 28 >= 20: # if having identifier, need to minus 10
                    energy_indices.append(i)
                else:
                    seq_indices.append(i)
    if dataset.find('HCV') != -1:
        for i in range(X_train.shape[1]):
            if i >= 952: #1326:
                energy_indices.append(i)
            else:
                if i % 28 >= 20: # if having identifier, need to minus 10
                    energy_indices.append(i)
                else:
                    seq_indices.append(i)
    if encoding == 'energy':
        X_train = X_train.iloc[:, energy_indices].copy()
        X_test = X_test.iloc[:, energy_indices].copy()
        if split == 3:
            X_val = X_val.iloc[:, energy_indices].copy()
    elif encoding == 'seq':
        X_train = X_train.iloc[:, seq_indices].copy()
        X_test = X_test.iloc[:, seq_indices].copy()
        if split == 3:
            X_val = X_val.iloc[:, seq_indices].copy()
    
    X_train = scale(X_train)
    X_test = scale(X_test)
    if split == 3:
        X_val = scale(X_val)

    if model == 'logistic_regression':
        from sklearn import linear_model
        lg = linear_model.LogisticRegression(C = 1, max_iter = 500)
        prob, acc = train_test(lg, X_train, y_train, X_test, y_test)
        print('Test Accuracy:{:.4f}'.format(acc))
        np.savetxt(os.path.join(save, 'logits_' + model + '_' + str(dataset) + '_' + encoding), prob)
    elif model == 'random_forest':
        av_acc = 0
        accs = []
        for i in range(20):
            from sklearn.ensemble import RandomForestClassifier
            rf = RandomForestClassifier()
            prob, acc = train_test(rf, X_train, y_train, X_test, y_test)
            av_acc += acc
            accs.append(acc)
            np.savetxt(os.path.join(save, 'logits_' + model + '_' + str(dataset) + '_' + encoding), prob)
        av_acc = av_acc / 20
        print('Test Accuracy:{:.4f}'.format(av_acc))
    elif model == 'decision_tree':
        av_acc = 0
        for i in range(20):
            from sklearn.tree import DecisionTreeClassifier
            dt = DecisionTreeClassifier(criterion='entropy')
            prob, acc = train_test(dt, X_train, y_train, X_test, y_test)
            av_acc += acc
        av_acc = av_acc / 20
        print('Test Accuracy:{:.4f}'.format(av_acc))
        np.savetxt(os.path.join(save, 'logits_' + model + '_' + str(dataset) + '_' + encoding), prob)
        logit_path = os.path.join(save, 'logits_' + model + '_' + str(dataset) + '_' + encoding)
    elif model == 'svm':
        from sklearn import svm
        svmsvc = svm.SVC(C = 1, probability=True)
        prob, acc = train_test(svmsvc, X_train, y_train, X_test, y_test)
        print('Test Accuracy:{:.4f}'.format(acc))
        np.savetxt(os.path.join(save, 'logits_' + model + '_' + str(dataset) + '_' + encoding), prob)
        logit_path = os.path.join(save, 'logits_' + model + '_' + str(dataset) + '_' + encoding)
    elif model == 'ann':
        import tensorflow as tf
        from tensorflow import keras
        os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
        if mode == 'train':
            accs = []
            dropout_list = [0.5,0.4,0.3,0.2,0.1,0.05,0.01]
            learning_rate = [0.01,0.05,1e-3,5e-3,1e-4,5e-4]
            combinations = []
            test_accs = []
            for dropout in dropout_list:
                for lr in learning_rate:
                    print('dropout: {}; learning rate: {}'.format(dropout, lr))
                    combinations.append([dropout, lr])
                    n_class = 2
                    ann = keras.Sequential([keras.layers.Dense(1024, activation=tf.nn.relu),
                                              keras.layers.Dropout(dropout, input_shape = (1024,)),
                                              keras.layers.Dense(n_class, activation=tf.nn.softmax)])

                    ann.compile(optimizer=tf.train.AdamOptimizer(learning_rate = lr),loss='sparse_categorical_crossentropy',metrics=['accuracy'])
                    if split == 2:
                        prob, acc = train_test_ann(ann, n_class, X_train, y_train, X_test, y_test)
                    elif split == 3:
                        prob, acc, test_prob, test_acc = train_test_ann_split(ann, n_class, X_train, y_train,
                                                                       X_test, y_test,
                                                                        X_val, y_val)
                    np.savetxt(os.path.join(save, 'logits_val_' + model + '_' + str(dataset) + '_' + encoding +
                                           '_dropout_' + str(dropout) + '_lr_' + str(lr)), prob)
                    np.savetxt(os.path.join(save, 'logits_test_' + model + '_' + str(dataset) + '_' + encoding +
                                           '_dropout_' + str(dropout) + '_lr_' + str(lr)), test_prob)
                    accs.append(acc)
                    test_accs.append(test_acc)

            print('Validation Accuracy:{:.4f}'.format(max(accs)))
            i=np.argmax(np.array(accs))
            print('Test Accuracy:{:.4f}'.format(test_accs[i]))
            print('Dropout: {:f}; Learning Rate: {:f}'.format(combinations[i][0], combinations[i][1]))
            logit_path = os.path.join(save, 'logits_test_' + model + '_' + str(dataset) + '_' + encoding +
                                           '_dropout_' + str(combinations[i][0]) + '_lr_' + str(combinations[i][1]))
        elif mode == 'test':
            n_class = 2
            ann = keras.Sequential([keras.layers.Dense(1024, activation=tf.nn.relu),
                                          keras.layers.Dropout(dropout, input_shape = (1024,)),
                                          keras.layers.Dense(n_class, activation=tf.nn.softmax)])

            ann.compile(optimizer=tf.train.AdamOptimizer(learning_rate = lr),loss='sparse_categorical_crossentropy',metrics=['accuracy'])
            if split == 2:
                prob, acc = train_test_ann(ann, n_class, X_train, y_train, X_test, y_test)
            elif split == 3:
                prob, acc, test_prob, test_acc = train_test_ann_split(ann, n_class, X_train, y_train,
                                                               X_test, y_test,
                                                                X_val, y_val)
            np.savetxt(os.path.join(save, 'logits_test_' + model + '_' + str(dataset) + '_' + encoding +
                                   '_dropout_' + str(dropout) + '_lr_' + str(lr)), test_prob)
        

In [ ]:
for enco in ['complete','seq']:
    for data in ['TEV_all_10_ang_aa_energy_7_energyedge_5_hbond_noProtID']:
        for model in ['logistic_regression','random_forest','decision_tree','svm','ann']:
            trainSeqOnly(data, model = model, save = '/scratch/cl1205/ml-cleavage/outputs/tev_20220623_tune')

#### trisplit, train test and validation TEV

In [7]:
# trisplit
for enco in ['complete','seq','energy']:
    for data in ['TEV_all_10_ang_aa_energy_7_energyedge_5_hbond_noProtID']:
        for model in ['logistic_regression']:
            print(enco)
            trainSeqOnly(data, model = model, encoding = enco, split=3,
                         save = '/scratch/cl1205/ml-cleavage/outputs/tev_noProtID_trisplit_20220629')

complete
Test Accuracy:0.8232
seq
Test Accuracy:0.8177
energy
Test Accuracy:0.8085


In [8]:
for enco in ['complete','seq','energy']:
    for data in ['TEV_all_10_ang_aa_energy_7_energyedge_5_hbond_noProtID']:
        for model in ['random_forest', 'decision_tree', 'svm']:
            print(enco)
            trainSeqOnly(data, model = model, encoding = enco, split=3,
                         save = '/scratch/cl1205/ml-cleavage/outputs/tev_noProtID_trisplit_20220629')

complete
Test Accuracy:0.7169
complete
Test Accuracy:0.4217
complete
Test Accuracy:0.7901
seq
Test Accuracy:0.8613
seq
Test Accuracy:0.8527
seq
Test Accuracy:0.8398
energy
Test Accuracy:0.7213
energy
Test Accuracy:0.5488
energy
Test Accuracy:0.7882


In [7]:
for enco in ['complete','seq','energy']:
    for data in ['TEV_all_10_ang_aa_energy_7_energyedge_5_hbond_noProtID']:
        for model in ['ann']:
            print(enco)
            trainSeqOnly(data, model = model, encoding = enco, split=3,
                         save = '/scratch/cl1205/ml-cleavage/outputs/tev_noProtID_trisplit_20220629')

complete
dropout: 0.5; learning rate: 0.01
Train on 4340 samples, validate on 542 samples
Epoch 1/10
4340/4340 [==============================] - 32s 7ms/sample - loss: 3.3455 - acc: 0.6979 - val_loss: 0.4580 - val_acc: 0.7694
Epoch 2/10
4340/4340 [==============================] - 32s 7ms/sample - loss: 0.5546 - acc: 0.7353 - val_loss: 0.5026 - val_acc: 0.7749
Epoch 3/10
4340/4340 [==============================] - 32s 7ms/sample - loss: 0.5676 - acc: 0.6581 - val_loss: 0.4356 - val_acc: 0.8044
Epoch 4/10
4340/4340 [==============================] - 32s 7ms/sample - loss: 0.5632 - acc: 0.6712 - val_loss: 0.4476 - val_acc: 0.8155
Epoch 5/10
542/542 [==============================] - 1s 2ms/sample - loss: 0.4669 - acc: 0.8026
dropout: 0.5; learning rate: 0.05
Train on 4340 samples, validate on 542 samples
Epoch 1/10
4340/4340 [==============================] - 32s 7ms/sample - loss: 28.1078 - acc: 0.6134 - val_loss: 0.9537 - val_acc: 0.6089
Epoch 2/10
4340/4340 [========================

TypeError: unsupported format string passed to list.__format__

In [ ]:
# HCV tripslit
for enco in ['complete', 'seq','energy']:
    for data in ['HCV_WT_10_ang_aa_energy_7_energyedge_5_hbond_noProtID',
                'HCV_A171T_10_ang_aa_energy_7_energyedge_5_hbond_noProtID',
                'HCV_D183A_10_ang_aa_energy_7_energyedge_5_hbond_noProtID',
                'HCV_Triple_10_ang_aa_energy_7_energyedge_5_hbond_noProtID',
                'HCV_all_10_ang_aa_energy_7_energyedge_5_hbond_noProtID']:
        for model in ['logistic_regression','random_forest','decision_tree','svm','ann']:
            print(enco)
            trainSeqOnly(data, model = model, encoding = enco, split=3,
                         save = '/scratch/cl1205/ml-cleavage/outputs/hcv_noProtID_trisplit_20220705')

complete
Test Accuracy:0.8612
complete
Test Accuracy:0.8307
complete
Test Accuracy:0.6585
complete
Test Accuracy:0.8925
complete
dropout: 0.5; learning rate: 0.01
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Train on 5873 samples, validate on 734 samples
Epoch 1/10
5873/5873 [==============================] - 16s 3ms/sample - loss: 1.5843 - acc: 0.7311 - val_loss: 0.5030 - val_acc: 0.7643
Epoch 2/10
5873/5873 [==============================] - 16s 3ms/sample - loss: 0.4981 - acc: 0.7494 - val_loss: 0.4032 - val_acc: 0.7616
Epoch 3/10
5873/5873 [==============================] - 16s 3ms/sample - loss: 0.5146 - acc: 0.7328 - val_loss: 0.4232 - val_acc: 0.7616
Epoch 4/10
5873/5873 [==============================] - 16s 3ms/sample - loss: 0.5328 - acc: 0.7306 - val_loss: 0.5551 - val_acc: 0.7616
Epoch 5/10
734/734 [==============================] - 0s 606us/sample - loss: 0.5486 - acc: 0.7616
dropout: 0.5; learning ra

#### decision tree change to run 20 times instead of single times, rerun all experiments for HCV and TEV

In [5]:
# HCV tripslit
# decision tree modified to 20 times
for enco in ['complete', 'seq','energy']:
    for data in ['HCV_WT_10_ang_aa_energy_7_energyedge_5_hbond_noProtID']:#,
#                 'HCV_A171T_10_ang_aa_energy_7_energyedge_5_hbond_noProtID',
#                 'HCV_D183A_10_ang_aa_energy_7_energyedge_5_hbond_noProtID',
#                 'HCV_Triple_10_ang_aa_energy_7_energyedge_5_hbond_noProtID',
#                 'HCV_all_10_ang_aa_energy_7_energyedge_5_hbond_noProtID']:
        for model in ['decision_tree']:
            print(data)
            print(enco)
            trainSeqOnly(data, model = model, encoding = enco, split=3,
                         save = '/scratch/cl1205/ml-cleavage/outputs/hcv_noProtID_trisplit_20220705')

HCV_WT_10_ang_aa_energy_7_energyedge_5_hbond_noProtID
complete
Test Accuracy:0.4735
HCV_WT_10_ang_aa_energy_7_energyedge_5_hbond_noProtID
seq
Test Accuracy:0.8766
HCV_WT_10_ang_aa_energy_7_energyedge_5_hbond_noProtID
energy
Test Accuracy:0.5373


#### decision tree 20 times, change criterion=entropy instead of default (gini)

In [7]:
# HCV tripslit
# decision tree modified to 20 times
for enco in ['complete', 'seq','energy']:
    for data in ['HCV_WT_10_ang_aa_energy_7_energyedge_5_hbond_noProtID']:#,
#                 'HCV_A171T_10_ang_aa_energy_7_energyedge_5_hbond_noProtID',
#                 'HCV_D183A_10_ang_aa_energy_7_energyedge_5_hbond_noProtID',
#                 'HCV_Triple_10_ang_aa_energy_7_energyedge_5_hbond_noProtID',
#                 'HCV_all_10_ang_aa_energy_7_energyedge_5_hbond_noProtID']:
        for model in ['decision_tree']:
            print(data)
            print(enco)
            trainSeqOnly(data, model = model, encoding = enco, split=3,
                         save = '/scratch/cl1205/ml-cleavage/outputs/hcv_noProtID_trisplit_20220705')

HCV_WT_10_ang_aa_energy_7_energyedge_5_hbond_noProtID
complete
Test Accuracy:0.8029
HCV_WT_10_ang_aa_energy_7_energyedge_5_hbond_noProtID
seq
Test Accuracy:0.8699
HCV_WT_10_ang_aa_energy_7_energyedge_5_hbond_noProtID
energy
Test Accuracy:0.8023


In [8]:
# HCV tripslit
# decision tree modified to 20 times
for enco in ['complete', 'seq','energy']:
    for data in [
                'HCV_A171T_10_ang_aa_energy_7_energyedge_5_hbond_noProtID',
                'HCV_D183A_10_ang_aa_energy_7_energyedge_5_hbond_noProtID',
                'HCV_Triple_10_ang_aa_energy_7_energyedge_5_hbond_noProtID',
                'HCV_all_10_ang_aa_energy_7_energyedge_5_hbond_noProtID']:
        for model in ['decision_tree']:
            print(data)
            print(enco)
            trainSeqOnly(data, model = model, encoding = enco, split=3,
                         save = '/scratch/cl1205/ml-cleavage/outputs/hcv_noProtID_trisplit_20220705')

HCV_A171T_10_ang_aa_energy_7_energyedge_5_hbond_noProtID
complete
Test Accuracy:0.7187
HCV_D183A_10_ang_aa_energy_7_energyedge_5_hbond_noProtID
complete
Test Accuracy:0.6759
HCV_Triple_10_ang_aa_energy_7_energyedge_5_hbond_noProtID
complete
Test Accuracy:0.6945
HCV_all_10_ang_aa_energy_7_energyedge_5_hbond_noProtID
complete
Test Accuracy:0.7668
HCV_A171T_10_ang_aa_energy_7_energyedge_5_hbond_noProtID
seq
Test Accuracy:0.9315
HCV_D183A_10_ang_aa_energy_7_energyedge_5_hbond_noProtID
seq
Test Accuracy:0.8448
HCV_Triple_10_ang_aa_energy_7_energyedge_5_hbond_noProtID
seq
Test Accuracy:0.8970
HCV_all_10_ang_aa_energy_7_energyedge_5_hbond_noProtID
seq
Test Accuracy:0.8770
HCV_A171T_10_ang_aa_energy_7_energyedge_5_hbond_noProtID
energy
Test Accuracy:0.7149
HCV_D183A_10_ang_aa_energy_7_energyedge_5_hbond_noProtID
energy
Test Accuracy:0.6842
HCV_Triple_10_ang_aa_energy_7_energyedge_5_hbond_noProtID
energy
Test Accuracy:0.6877
HCV_all_10_ang_aa_energy_7_energyedge_5_hbond_noProtID
energy
Test Acc

In [9]:
# TEV tripslit
# decision tree modified to 20 times
for enco in ['complete', 'seq','energy']:
    for data in ['TEV_all_10_ang_aa_energy_7_energyedge_5_hbond_noProtID']:
        for model in ['decision_tree']:
            print(data)
            print(enco)
            trainSeqOnly(data, model = model, encoding = enco, split=3,
                         save = '/scratch/cl1205/ml-cleavage/outputs/tev_noProtID_trisplit_20220629')

TEV_all_10_ang_aa_energy_7_energyedge_5_hbond_noProtID
complete
Test Accuracy:0.6750
TEV_all_10_ang_aa_energy_7_energyedge_5_hbond_noProtID
seq
Test Accuracy:0.8531
TEV_all_10_ang_aa_energy_7_energyedge_5_hbond_noProtID
energy
Test Accuracy:0.6703


In [ ]:
for data in ['TEV_all']:
    for model in ['ann']:
        trainSeqOnly(data, model = model, save = '/scratch/cl1205/ml-cleavage/outputs/tev_20220611')

In [ ]:
X_train, y_train, X_test, y_test = load_data('TEV_all_10_ang_aa_energy_7_energyedge_5_hbond', 2)

In [ ]:
X_train,shape

In [ ]:
# X_train.iloc[:,np.r_[0:30, 38:58, 66:86, 94:114, 122:142, 1242:1262, 1270:1290]]

In [ ]:
# base accuracy
baseAccs = []
classes = 2
labelorder = ['CLEAVED','UNCLEAVED']
for dataset in ['TEV_all']:
    X_train, y_train, X_test, y_test = load_data(dataset, classes)
    fakePositive = ['CLEAVED'] * y_test.shape[0]
    fakeNegative = ['UNCLEAVED'] * y_test.shape[0]

    baseAccs.append([dataset, np.mean(np.array(fakePositive) == y_test.to_numpy()), np.mean(np.array(fakeNegative) == y_test.to_numpy())])

In [ ]:
baseAccs

In [ ]:
for enco in ['complete','seq','energy']:
    for data in ['HCV_WT_10_ang_aa_energy_7_energyedge_5_hbond_noProtID', 
                 'HCV_A171T_10_ang_aa_energy_7_energyedge_5_hbond_noProtID', 
                 'HCV_D183A_10_ang_aa_energy_7_energyedge_5_hbond_noProtID', 
                 'HCV_Triple_10_ang_aa_energy_7_energyedge_5_hbond_noProtID', 
                 'HCV_all_10_ang_aa_energy_7_energyedge_5_hbond_noProtID']:
        for model in ['logistic_regression','random_forest','decision_tree','svm','ann']:
            print(model)
            print(enco)
            trainSeqOnly(data, model = model, encoding = enco, 
                         save = '/scratch/cl1205/ml-cleavage/outputs/hcv_noProtID_20220624')
            

In [ ]:
for enco in ['seq','energy']:
    for data in ['HCV_WT_10_ang_aa_energy_7_energyedge_5_hbond_noProtID', 
                 'HCV_A171T_10_ang_aa_energy_7_energyedge_5_hbond_noProtID', 
                 'HCV_D183A_10_ang_aa_energy_7_energyedge_5_hbond_noProtID', 
                 'HCV_Triple_10_ang_aa_energy_7_energyedge_5_hbond_noProtID', 
                 'HCV_all_10_ang_aa_energy_7_energyedge_5_hbond_noProtID']:
        for model in ['logistic_regression','random_forest','decision_tree','svm','ann']:
            print(model)
            print(enco)
            trainSeqOnly(data, model = model, encoding = enco, 
                         save = '/scratch/cl1205/ml-cleavage/outputs/hcv_noProtID_20220624')

In [ ]:
for enco in ['seq']:
    for data in ['HCV_all_10_ang_aa_energy_7_energyedge_5_hbond_noProtID']:
        for model in ['random_forest','decision_tree','svm','ann']:
            print(model)
            print(enco)
            trainSeqOnly(data, model = model, encoding = enco, 
                         save = '/scratch/cl1205/ml-cleavage/outputs/hcv_noProtID_20220624')

In [ ]:
for enco in ['energy']:
    for data in ['HCV_WT_10_ang_aa_energy_7_energyedge_5_hbond_noProtID', 
                 'HCV_A171T_10_ang_aa_energy_7_energyedge_5_hbond_noProtID', 
                 'HCV_D183A_10_ang_aa_energy_7_energyedge_5_hbond_noProtID', 
                 'HCV_Triple_10_ang_aa_energy_7_energyedge_5_hbond_noProtID', 
                 'HCV_all_10_ang_aa_energy_7_energyedge_5_hbond_noProtID']:
        for model in ['logistic_regression','random_forest','decision_tree','svm','ann']:
            print(model)
            print(enco)
            trainSeqOnly(data, model = model, encoding = enco, 
                         save = '/scratch/cl1205/ml-cleavage/outputs/hcv_noProtID_20220624')

In [ ]:
graph2mlData('TEV_all', 'seq')

In [ ]:
graph2mlData('TEV_all', 'flatten')

In [ ]:
# because no protID, codes should be modified that search features before and after 1316th column
for data in ['TEV_all_noProtID', 'TEV_all_10_ang_aa_energy_7_energyedge_5_hbond_noProtID']:
    for model in ['logistic_regression','random_forest','decision_tree','svm','ann']:
        trainSeqOnly(data, model = model, save = '/scratch/cl1205/ml-cleavage/outputs/tev_noProtID_20220618',
                    encoding = 'complete')

In [ ]:
for data in ['TEV_all_10_ang_aa_energy_7_energyedge_5_hbond_noProtID']:
    for model in ['logistic_regression','random_forest','decision_tree','svm','ann']:
        trainSeqOnly(data, model = model, save = '/scratch/cl1205/ml-cleavage/outputs/tev_noProtID_20220618',
                    encoding = 'energy')
        trainSeqOnly(data, model = model, save = '/scratch/cl1205/ml-cleavage/outputs/tev_noProtID_20220618',
                    encoding = 'seq')

In [ ]:
dataset = 'TEV_all_10_ang_aa_energy_7_energyedge_5_hbond_noProtID'
encoding = 'complete'
classes = 2
X_train, y_train, X_test, y_test = load_data(dataset, classes)
energy_indices = []
seq_indices = []
for i in range(X_train.shape[1]):
    if i >= 1316: #1326:
        energy_indices.append(i)
    else:
        if (i - 10) % 28 >= 20:
            energy_indices.append(i)
        else:
            seq_indices.append(i)
if encoding == 'energy':
    X_train = X_train.iloc[:, energy_indices].copy()
    X_test = X_test.iloc[:, energy_indices].copy()
elif encoding == 'seq':
    X_train = X_train.iloc[:, seq_indices].copy()
    X_test = X_test.iloc[:, seq_indices].copy()
print(X_train.shape)

X_train = scale(X_train)
X_test = scale(X_test)

In [ ]:
from sklearn import linear_model
lg = linear_model.LogisticRegression(C = 1, max_iter = 500)
prob, acc = train_test(lg, X_train, y_train, X_test, y_test)
print('Test Accuracy:{:.4f}'.format(acc))

In [ ]:
from sklearn import linear_model
lg = linear_model.LogisticRegression(C = 1, max_iter = 500)
lg.fit(X_train, y_train)
pre = lg.predict(X_test)


In [ ]:
pre

In [ ]:
y_test.to_numpy().flatten()

In [ ]:
prob = lg.predict_proba(X_test)
acc = np.mean(pre == y_test.to_numpy().flatten())

In [ ]:
acc

# Other Metrics for ML models

In [7]:
from sklearn.metrics import roc_curve, auc, average_precision_score
def getAUC(true, proba):
    from sklearn.metrics import roc_curve, auc, average_precision_score
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    # cleaved = 0
    fpr, tpr, _ = roc_curve(true, proba[:,0], pos_label=0)
    roc_auc = auc(fpr, tpr) #only for binary class
    return '{:.4f}'.format(roc_auc)
def precision_recall(true, predict, pos_label=0):
    # true positive, false negative, false positive
    tp = np.sum(true[np.where(predict == pos_label)] == pos_label)
    fp = np.sum(true[np.where(predict == pos_label)] != pos_label)
    fn = np.sum(true[np.where(predict != pos_label)] == pos_label)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2 * precision * recall / (precision + recall)
    ap = average_precision_score(true, predict)
    return precision, recall, f1, ap

In [8]:
def softmax(x,n):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x)
    if n == 1:
        return e_x / e_x.sum(axis=n).reshape(-1,1)
    else:
        return e_x / e_x.sum(axis=n)

In [9]:
def read_true_test_label(data, mode='split'):
#     if data == 'WT':
#         all_df = pd.read_csv('graph/classifications/HCV.txt', delimiter='\t')
#     elif data == 'Triple':
#         all_df = pd.read_csv('graph/classifications/HCV_R170K_A171T_D183A.txt', delimiter='\t')
#     else:
#         all_df = pd.read_csv("graph/classifications/HCV_" + data + '.txt', delimiter='\t')
    
    #     true = all_df.iloc[test_index,:]
    if mode == 'split':
        y = pkl.load(open("/scratch/cl1205/protease-gcnn-pytorch/data/ind." + '_'.join(data.split('_')[0:2]) + "_binary_10_ang_aa_energy_7_energyedge_5_hbond.y", "rb"))
        try:
            y = y.numpy()
        except AttributeError:
            pass

        test_index = np.loadtxt("/scratch/cl1205/protease-gcnn-pytorch/data/ind." + '_'.join(data.split('_')[0:2]) + "_binary_10_ang_aa_energy_7_energyedge_5_hbond.trisplit.test.index", dtype=int)
        true = np.argmax(y,axis=1)[np.sort(test_index)]
    elif mode == 'individual':
        y = pkl.load(open("/scratch/cl1205/ml-cleavage/Data/y_test_" + data, "rb"))
        try:
            y = y.numpy()
        except AttributeError:
            pass
        true = transform(y, 2)
#     if data == 'WT':
#         true = true[:-1]
    return true

In [10]:
def calMetrics(data_name, test_name, model, path='tev_noProtID_20220624',encoding='complete', mode = 'split', dropout=0, lr=0):
    if model != 'ann':
        logits = np.loadtxt("/scratch/cl1205/ml-cleavage/outputs/" + path + "/logits_" + model + "_" + data_name +
                       "_" + encoding)
    else:
        logits = np.loadtxt("/scratch/cl1205/ml-cleavage/outputs/" + path + "/logits_test_" + model + "_" + data_name +
                       "_" + encoding + '_dropout_' + str(dropout) + '_lr_' + str(lr))
        print("cp /scratch/cl1205/ml-cleavage/outputs/" + path + "/logits_test_" + model + "_" + data_name +
                       "_" + encoding + '_dropout_' + str(dropout) + '_lr_' + str(lr) + " ./")
    true = read_true_test_label(test_name, mode)
    predict = np.argmax(logits,axis=1)
    if model == 'ann':
        proba = softmax(logits,1)
    else:
        proba = logits
#     if data_name.find('HCV_WT') != -1:
#         proba = proba[:-1,:]
#         predict = predict[:-1]
#     elif data_name.find('HCV_all') != -1:
#         proba = proba[:-2,:]
#         predict = predict[:-2]
    acc = np.mean(np.array([predict[i] == true[i] for i in range(predict.shape[0])]))
#     print('acc: {:.4f}'.format(acc))
    
    # auc
    fpr = dict()
    tpr = dict()
    roc_auc = dict()

    fpr, tpr, _ = roc_curve(true, proba[:,0], pos_label=0)
    roc_auc = auc(fpr, tpr) #only for binary class
#     print('auc: {:.4f}'.format(roc_auc))

    # precision, recall, f1
    precision, recall, f1, ap = precision_recall(true, predict, pos_label=0)
#     print('precision: {:.4f}'.format(precision))
#     print('recall: {:.4f}'.format(recall))
#     print('f1: {:.4f}'.format(f1))
#     print('ap: {:.4f}'.format(ap))
    return acc, roc_auc, precision, recall, f1, ap

In [40]:
def predictLabel(data_name, test_name, model, path='tev_noProtID_20220624',encoding='complete', dropout=0, lr=0):
    if model != 'ann':
        logits = np.loadtxt("/scratch/cl1205/ml-cleavage/outputs/" + path + "/logits_" + model + "_" + data_name +
                       "_" + encoding)
    else:
        logits = np.loadtxt("/scratch/cl1205/ml-cleavage/outputs/" + path + "/logits_test_" + model + "_" + data_name +
                       "_" + encoding + '_dropout_' + str(dropout) + '_lr_' + str(lr))
    
    predict = np.argmax(logits,axis=1)
    if model == 'ann':
        proba = softmax(logits,1)
    else:
        proba = logits
    probabilities = []
    for k in range(proba.shape[0]):
        if proba[k,0] > proba[k,1]:
            probabilities.append(proba[k,0])
        else:
            probabilities.append(proba[k,1])

    results = []
    for l in predict:
        if l == 1:
            results.append('UNCLEAVED')
        elif l == 0:
            results.append('CLEAVED')
    sequences = pkl.load(open("/scratch/cl1205/ml-cleavage/Data/X_test_" + test_name, "rb"))
    sequences = sequences.index.values
    trueLabels = pkl.load(open("/scratch/cl1205/ml-cleavage/Data/y_test_" + test_name, "rb"))
    df = pd.DataFrame({'Sequence': sequences, 'Model': [model]*len(sequences), 'Feature': [encoding]*len(sequences),
                       'Probabilities': probabilities, 'Result': results,
                      'TrueLabel': trueLabels[0].values})
    return df

In [ ]:
for enco in ['complete', 'seq', 'energy']:
    for m in ['logistic_regression', 'random_forest', 'decision_tree', 'svm', 'ann']:
        print(m)
        print(enco)
        calMetrics('TEV_all_10_ang_aa_energy_7_energyedge_5_hbond_noProtID', m, encoding=enco)

In [74]:
results = [['data','model','feature','accuracy','auc','precision','recall','f1','ap']]

params = [[0.005,0.01],[0.05,0.3],[0.01,0.01]]
# print('TEV_all')
for i, enco in enumerate(['complete', 'seq', 'energy']):
    for m in ['logistic_regression', 'random_forest', 'decision_tree', 'svm', 'ann']:
#         print(m)
#         print(enco)
        if m == 'ann':
            accuracy, auc_score, precision, recall, f1, ap = calMetrics('TEV_all_10_ang_aa_energy_7_energyedge_5_hbond_noProtID', 
                       m, path='tev_noProtID_trisplit_20220629',
                       encoding=enco, lr=params[i][0], dropout=params[i][1])
            results.append(['TEV_all', m, enco, accuracy, auc_score, precision, recall, f1, ap])
        else:
            accuracy, auc_score, precision, recall, f1, ap = calMetrics('TEV_all_10_ang_aa_energy_7_energyedge_5_hbond_noProtID', 
                       m, path='tev_noProtID_trisplit_20220629', encoding=enco)
            results.append(['TEV_all', m, enco, accuracy, auc_score, precision, recall, f1, ap])

params = [[0.0005, 0.3], [0.0005, 0.4], [0.0005, 0.1]]
# print('HCV_WT')
for i, enco in enumerate(['complete', 'seq', 'energy']):
    for m in ['logistic_regression', 'random_forest', 'decision_tree', 'svm', 'ann']:
#         print(m)
#         print(enco)
        if m == 'ann':
            accuracy, auc_score, precision, recall, f1, ap = calMetrics('HCV_WT_10_ang_aa_energy_7_energyedge_5_hbond_noProtID', 
                       m, path='hcv_noProtID_trisplit_20220705',
                       encoding=enco, lr=params[i][0], dropout=params[i][1])
            results.append(['HCV_WT', m, enco, accuracy, auc_score, precision, recall, f1, ap])
        else:
            accuracy, auc_score, precision, recall, f1, ap = calMetrics('HCV_WT_10_ang_aa_energy_7_energyedge_5_hbond_noProtID', 
                       m, path='hcv_noProtID_trisplit_20220705', encoding=enco)
            results.append(['HCV_WT', m, enco, accuracy, auc_score, precision, recall, f1, ap])
            
params = [[0.001, 0.01], [0.0005, 0.4], [0.0005, 0.01]]
# print('HCV_A171T')
for i, enco in enumerate(['complete', 'seq', 'energy']):
    for m in ['logistic_regression', 'random_forest', 'decision_tree', 'svm', 'ann']:
#         print(m)
#         print(enco)
        if m == 'ann':
            accuracy, auc_score, precision, recall, f1, ap = calMetrics('HCV_A171T_10_ang_aa_energy_7_energyedge_5_hbond_noProtID', 
                       m, path='hcv_noProtID_trisplit_20220705',
                       encoding=enco, lr=params[i][0], dropout=params[i][1])
            results.append(['HCV_A171T', m, enco, accuracy, auc_score, precision, recall, f1, ap])
        else:
            accuracy, auc_score, precision, recall, f1, ap = calMetrics('HCV_A171T_10_ang_aa_energy_7_energyedge_5_hbond_noProtID', 
                       m, path='hcv_noProtID_trisplit_20220705', encoding=enco)
            results.append(['HCV_A171T', m, enco, accuracy, auc_score, precision, recall, f1, ap])
            
params = [[0.01, 0.01], [0.0005, 0.5], [0.01, 0.3]]
# print('HCV_D183A')
for i, enco in enumerate(['complete', 'seq', 'energy']):
    for m in ['logistic_regression', 'random_forest', 'decision_tree', 'svm', 'ann']:
#         print(m)
#         print(enco)
        if m == 'ann':
            accuracy, auc_score, precision, recall, f1, ap = calMetrics('HCV_D183A_10_ang_aa_energy_7_energyedge_5_hbond_noProtID', 
                       m, path='hcv_noProtID_trisplit_20220705',
                       encoding=enco, lr=params[i][0], dropout=params[i][1])
            results.append(['HCV_D183A', m, enco, accuracy, auc_score, precision, recall, f1, ap])
        else:
            accuracy, auc_score, precision, recall, f1, ap = calMetrics('HCV_D183A_10_ang_aa_energy_7_energyedge_5_hbond_noProtID', 
                       m, path='hcv_noProtID_trisplit_20220705', encoding=enco)
            results.append(['HCV_D183A', m, enco, accuracy, auc_score, precision, recall, f1, ap])
            
params = [[0.001, 0.05], [0.0005, 0.1], [0.001, 0.01]]
# print('HCV_Triple')
for i, enco in enumerate(['complete', 'seq', 'energy']):
    for m in ['logistic_regression', 'random_forest', 'decision_tree', 'svm', 'ann']:
#         print(m)
#         print(enco)
        if m == 'ann':
            accuracy, auc_score, precision, recall, f1, ap = calMetrics('HCV_Triple_10_ang_aa_energy_7_energyedge_5_hbond_noProtID', 
                       m, path='hcv_noProtID_trisplit_20220705',
                       encoding=enco, lr=params[i][0], dropout=params[i][1])
            results.append(['HCV_Triple', m, enco, accuracy, auc_score, precision, recall, f1, ap])
        else:
            accuracy, auc_score, precision, recall, f1, ap = calMetrics('HCV_Triple_10_ang_aa_energy_7_energyedge_5_hbond_noProtID', 
                       m, path='hcv_noProtID_trisplit_20220705', encoding=enco)
            results.append(['HCV_Triple', m, enco, accuracy, auc_score, precision, recall, f1, ap])

params = [[0.0001, 0.01], [0.0001, 0.2], [0.0001, 0.1]]

for i, enco in enumerate(['complete', 'seq', 'energy']):
    for m in ['logistic_regression', 'random_forest', 'decision_tree', 'svm', 'ann']:
#         print('HCV_all')
#         print(m)
#         print(enco)
        if m == 'ann':
            accuracy, auc_score, precision, recall, f1, ap = calMetrics('HCV_all_10_ang_aa_energy_7_energyedge_5_hbond_noProtID', 
                       m, path='hcv_noProtID_trisplit_20220705',
                       encoding=enco, lr=params[i][0], dropout=params[i][1])
            results.append(['HCV_all', m, enco, accuracy, auc_score, precision, recall, f1, ap])
        else:
            accuracy, auc_score, precision, recall, f1, ap = calMetrics('HCV_all_10_ang_aa_energy_7_energyedge_5_hbond_noProtID', 
                       m, path='hcv_noProtID_trisplit_20220705', encoding=enco)
            results.append(['HCV_all', m, enco, accuracy, auc_score, precision, recall, f1, ap])

df = pd.DataFrame(results[1:], columns=results[0])

cp /scratch/cl1205/ml-cleavage/outputs/tev_noProtID_trisplit_20220629/logits_test_ann_TEV_all_10_ang_aa_energy_7_energyedge_5_hbond_noProtID_complete_dropout_0.01_lr_0.005 ./
cp /scratch/cl1205/ml-cleavage/outputs/tev_noProtID_trisplit_20220629/logits_test_ann_TEV_all_10_ang_aa_energy_7_energyedge_5_hbond_noProtID_seq_dropout_0.3_lr_0.05 ./
cp /scratch/cl1205/ml-cleavage/outputs/tev_noProtID_trisplit_20220629/logits_test_ann_TEV_all_10_ang_aa_energy_7_energyedge_5_hbond_noProtID_energy_dropout_0.01_lr_0.01 ./
cp /scratch/cl1205/ml-cleavage/outputs/hcv_noProtID_trisplit_20220705/logits_test_ann_HCV_WT_10_ang_aa_energy_7_energyedge_5_hbond_noProtID_complete_dropout_0.3_lr_0.0005 ./
cp /scratch/cl1205/ml-cleavage/outputs/hcv_noProtID_trisplit_20220705/logits_test_ann_HCV_WT_10_ang_aa_energy_7_energyedge_5_hbond_noProtID_seq_dropout_0.4_lr_0.0005 ./
cp /scratch/cl1205/ml-cleavage/outputs/hcv_noProtID_trisplit_20220705/logits_test_ann_HCV_WT_10_ang_aa_energy_7_energyedge_5_hbond_noProtID_en

In [67]:
df.to_csv('MetricSummary_ML_AUC_Precision_Recall_F1_AP.csv', index=None)

### test tev/hcv design

In [37]:
testset = 'test_tev_design_20220912_binary_10_ang_aa_energy_7_coord_energyedge_5_hbond'
X_test = pkl.load(open('../Data/X_test_' + testset,'rb'))
y_test = pkl.load(open('../Data/y_test_' + testset,'rb'))

In [44]:
dataset = 'test_tev_design_20220912_p1pA_binary_10_ang_aa_energy_7_coord_energyedge_5_hbond'
pkl.dump(X_test, open('/scratch/cl1205/ml-cleavage/Data/X_test_{}'.format(dataset),'wb'))
pkl.dump(y_test, open('/scratch/cl1205/ml-cleavage/Data/y_test_{}'.format(dataset),'wb'))

In [21]:
results = [['data','model','feature','accuracy','auc','precision','recall','f1','ap']]
testdata = 'test_20220922_tev_design_dual_directions_cleavage_binary_10_ang_aa_energy_7_energyedge_5_hbond'
data = 'TEV_all_10_ang_aa_energy_7_energyedge_5_hbond_noProtID'
params = [[0.005,0.01],[0.05,0.3],[0.01,0.01]]

# print('TEV_all')
for i, enco in enumerate(['complete', 'seq', 'energy']):
    for m in ['logistic_regression', 'random_forest', 'decision_tree', 'svm', 'ann']:
        
        print(m)
        print(enco)
        if m == 'ann':
            trainSeqOnly(data, model = m, encoding = enco, split=3, testset = testdata,
                             save = '/scratch/cl1205/ml-cleavage/outputs/tev_design_dual_direction_cleavage_20220925', 
                         mode='test', lr=params[i][0], dropout=params[i][1])
            if enco == 'complete':
                accuracy, auc_score, precision, recall, f1, ap = calMetrics(testdata, testdata,
                           m, path='tev_design_dual_direction_cleavage_20220925', mode='individual',
                           encoding=enco, lr=params[i][0], dropout=params[i][1])
            else:
                accuracy, auc_score, precision, recall, f1, ap = calMetrics(data, testdata,
                           m, path='tev_design_dual_direction_cleavage_20220925', mode='individual',
                           encoding=enco, lr=params[i][0], dropout=params[i][1])
            results.append(['TEV_all', m, enco, accuracy, auc_score, precision, recall, f1, ap])
        else:
            trainSeqOnly(data, model = m, encoding = enco, split=3, testset = testdata,
                            save = '/scratch/cl1205/ml-cleavage/outputs/tev_design_dual_direction_cleavage_20220925')
            accuracy, auc_score, precision, recall, f1, ap = calMetrics(data, testdata,
                       m, path='tev_design_dual_direction_cleavage_20220925', mode='individual', encoding=enco)
            results.append(['TEV_all', m, enco, accuracy, auc_score, precision, recall, f1, ap])
df = pd.DataFrame(results[1:], columns=results[0])
df.to_csv('MetricSummary_TEV_design_dual_direction_MLmodels.csv', index=None)

logistic_regression
complete
path of data:
../Data/X_train_TEV_all_10_ang_aa_energy_7_energyedge_5_hbond_noProtID_trisplit_0.8
../Data/X_test_TEV_all_10_ang_aa_energy_7_energyedge_5_hbond_noProtID_trisplit_0.1
../Data/X_val_TEV_all_10_ang_aa_energy_7_energyedge_5_hbond_noProtID_trisplit_0.1
../Data/y_train_TEV_all_10_ang_aa_energy_7_energyedge_5_hbond_noProtID_trisplit_0.8
../Data/y_test_TEV_all_10_ang_aa_energy_7_energyedge_5_hbond_noProtID_trisplit_0.1
../Data/y_val_TEV_all_10_ang_aa_energy_7_energyedge_5_hbond_noProtID_trisplit_0.1
Test Accuracy:0.3667
random_forest
complete
path of data:
../Data/X_train_TEV_all_10_ang_aa_energy_7_energyedge_5_hbond_noProtID_trisplit_0.8
../Data/X_test_TEV_all_10_ang_aa_energy_7_energyedge_5_hbond_noProtID_trisplit_0.1
../Data/X_val_TEV_all_10_ang_aa_energy_7_energyedge_5_hbond_noProtID_trisplit_0.1
../Data/y_train_TEV_all_10_ang_aa_energy_7_energyedge_5_hbond_noProtID_trisplit_0.8
../Data/y_test_TEV_all_10_ang_aa_energy_7_energyedge_5_hbond_noProtI

OSError: /scratch/cl1205/ml-cleavage/outputs/tev_design_dual_direction_cleavage_20220925/logits_test_ann_test_20220922_tev_design_dual_directions_cleavage_binary_10_ang_aa_energy_7_energyedge_5_hbond_complete_dropout_0.01_lr_0.005 not found.

In [26]:
df.to_csv('MetricSummary_TEV_design_dual_direction_MLmodels.csv', index=None)

In [25]:
df

,data,model,feature,accuracy,auc,precision,recall,f1,ap
0,TEV_all,logistic_regression,complete,0.366667,0.921364,1.000000,0.136364,0.240000,0.296296
1,TEV_all,random_forest,complete,0.606667,0.620000,0.814815,0.600000,0.691099,0.326449
2,TEV_all,decision_tree,complete,0.666667,0.693182,0.875000,0.636364,0.736842,0.388095
3,TEV_all,svm,complete,0.733333,0.548182,0.743056,0.972727,0.842520,0.284167
4,TEV_all,ann,complete,0.346667,0.424773,0.928571,0.118182,0.209677,0.286262
5,TEV_all,ann,seq,0.333333,0.113636,0.571429,0.363636,0.444444,0.231250
6,TEV_all,ann,energy,0.740000,0.360000,0.738255,1.000000,0.849421,0.285000


In [25]:
tmp_df = pd.DataFrame()
testdata = 'test_20220922_tev_design_dual_directions_cleavage_binary_10_ang_aa_energy_7_energyedge_5_hbond'
data = 'TEV_all_10_ang_aa_energy_7_energyedge_5_hbond_noProtID'
params = [[0.005,0.01],[0.05,0.3],[0.01,0.01]]

for i, enco in enumerate(['complete', 'seq', 'energy']):
    for m in ['logistic_regression', 'random_forest', 'decision_tree', 'svm', 'ann']:
        if m == 'ann':
            df = predictLabel(data, testdata, m, path='tev_design_dual_direction_cleavage_20220925',
                  encoding=enco, dropout=params[i][1], lr=params[i][0])
        else:
            df = predictLabel(data, testdata, m, path='tev_design_dual_direction_cleavage_20220925',
                  encoding=enco)
        tmp_df = pd.concat((tmp_df, df), axis=0)

In [27]:
tmp_df.to_csv('TEV_design_dual_direction_MLmodels_report.csv', index=None)